Celem tego skryptu jest przygotowanie danych do dalszej analizy.
Skrypt korzysta z danych pobranych z API ENTSO-E.
Następnie dane są przetwarzane i zapisywane w formacie CSV do dalszej analizy.
Dane z ENTSO-E są w postaci dziennej, a niektóre nawet co 15 minut.
Skrypt agreguje je do postaci kwartalnej, aby ułatwić dalszą analizę.

In [1]:
pip install tabulate matplotlib seaborn scikit-learn statsmodels

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import os

print("Start Skryptu 2: Generowanie Danych")

# --- KONFIGURACJA ---
SOURCE_DIR = 'dane_zrodlowe_API'
DEST_DIR = 'dane_po_transformacji'
OUTPUT_FILENAME = 'dane_miesieczne_DE.csv'
OUTPUT_PATH = os.path.join(DEST_DIR, OUTPUT_FILENAME)

if not os.path.exists(DEST_DIR):
    os.makedirs(DEST_DIR)

# --- FUNKCJE POMOCNICZE ---

def clean_entsoe_index(df):
    """Naprawa indeksu czasowego."""
    df.index = pd.to_datetime(df.index, utc=True, errors='coerce')
    df = df.dropna(how='all')
    df = df.sort_index()
    return df

# --- GŁÓWNA LOGIKA ETL ---

def run_etl_process():
    
    # 0. DEFINICJE
    RES_KEYWORDS = ['Wind', 'Solar', 'Hydro', 'Biomass', 'Geothermal', 'Waste', 'Other renewable']
    FOSSIL_KEYWORDS = ['Fossil', 'Lignite', 'Hard Coal', 'Gas', 'Oil', 'Coal', 'Peat']
    
    # --- 1. PRZETWARZANIE GENERACJI ---
    print("\n1. Przetwarzanie danych generacji...")
    df_gen = pd.read_csv(os.path.join(SOURCE_DIR, 'raw_generation_de.csv'), index_col=0, low_memory=False)
    df_gen = clean_entsoe_index(df_gen)
    df_gen = df_gen.apply(pd.to_numeric, errors='coerce').fillna(0)
    df_gen = df_gen.dropna(axis=1, how='all')

    # Sumowanie
    cols_res = [c for c in df_gen.columns if any(x in c for x in RES_KEYWORDS)]
    cols_fossil = [c for c in df_gen.columns if any(x in c for x in FOSSIL_KEYWORDS)]
    
    df_gen['Total_Res'] = df_gen[cols_res].sum(axis=1)
    df_gen['Total_Fossil'] = df_gen[cols_fossil].sum(axis=1)
    df_gen['Total_All'] = df_gen.sum(axis=1)

    # AGREGACJA MIESIĘCZNA
    df_gen_m = df_gen[['Total_Res', 'Total_Fossil', 'Total_All']].resample('ME').sum()
    
    df_features = pd.DataFrame(index=df_gen_m.index)
    df_features['res_share'] = df_gen_m['Total_Res'] / df_gen_m['Total_All']
    df_features['fossil_share'] = df_gen_m['Total_Fossil'] / df_gen_m['Total_All']

    # --- 2. PRZETWARZANIE OBCIĄŻENIA ---
    print("2. Przetwarzanie danych obciążenia (konwersja na MWh)...")
    df_load = pd.read_csv(os.path.join(SOURCE_DIR, 'raw_load_de.csv'), index_col=0)
    df_load = clean_entsoe_index(df_load)
    df_load.columns = ['Actual_Load']
    
    df_forecast = pd.read_csv(os.path.join(SOURCE_DIR, 'raw_load_forecast_de.csv'), index_col=0)
    df_forecast = clean_entsoe_index(df_forecast)
    df_forecast.columns = ['Forecast_Load']
    
    df_load_merged = pd.concat([df_load, df_forecast], axis=1).dropna()
    df_load_merged['PctError'] = abs(df_load_merged['Actual_Load'] - df_load_merged['Forecast_Load']) / df_load_merged['Actual_Load']

    # AGREGACJA MIESIĘCZNA
    # Dzielimy sumę przez 4, aby przejść z mocy 15-min (MW) na energię (MWh)
    df_features['energy_consumption'] = df_load_merged['Actual_Load'].resample('ME').sum() / 4
    
    # Stabilność zostaje bez zmian (to średnia z procentów)
    df_features['system_stability'] = df_load_merged['PctError'].resample('ME').mean()

    # --- 3. PRZETWARZANIE CEN ---
    print("3. Przetwarzanie danych cenowych...")
    df_prices = pd.read_csv(os.path.join(SOURCE_DIR, 'raw_prices_de_combined.csv'), index_col=0)
    df_prices = clean_entsoe_index(df_prices)
    df_prices.columns = ['Price_EUR_MWh']
    df_prices['Price_EUR_MWh'] = pd.to_numeric(df_prices['Price_EUR_MWh'], errors='coerce')
    
    # AGREGACJA MIESIĘCZNA
    df_features['market_price'] = df_prices['Price_EUR_MWh'].resample('ME').mean()
    df_features['price_volatility'] = df_prices['Price_EUR_MWh'].resample('ME').std()

    # --- 4. DODANIE FLAG I FORMATOWANIE ---
    print("4. Formatowanie kolumn (Date, Month, Year, ID)...")
    
    df_final = df_features.dropna()

    # Flaga kryzysowa
    df_final['crisis_flag'] = 0
    mask_covid = (df_final.index >= '2020-03-01') & (df_final.index <= '2021-06-30')
    mask_energy = (df_final.index >= '2021-09-01') & (df_final.index <= '2023-12-31')
    df_final.loc[mask_covid | mask_energy, 'crisis_flag'] = 1

    # --- TWORZENIE STRUKTURY DOCELOWEJ ---
    df_final = df_final.reset_index()
    df_final = df_final.rename(columns={'index': 'date'})

    df_final['month'] = df_final['date'].dt.month
    df_final['year'] = df_final['date'].dt.year
    df_final['period_id'] = np.arange(1, len(df_final) + 1)

    cols_order = [
        'date', 'month', 'year', 'period_id',
        'res_share', 'fossil_share', 
        'market_price', 'price_volatility', 
        'energy_consumption', 'system_stability',
        'crisis_flag'
    ]
    
    df_final = df_final[cols_order]

    # Zapis
    df_final.to_csv(OUTPUT_PATH, index=False)
    
    print("\n--- PODGLĄD DANYCH (Pierwsze 5 wierszy) ---")
    # Formatujemy wyświetlanie liczb float dla czytelności
    with pd.option_context('display.float_format', '{:,.2f}'.format):
        print(df_final.head().to_markdown(index=False))
    
    print(f"\nZapisano plik: {OUTPUT_FILENAME}")
    print(f"Lokalizacja: {OUTPUT_PATH}")

if __name__ == '__main__':
    run_etl_process()

Start Skryptu 2: Generowanie Danych

1. Przetwarzanie danych generacji...
2. Przetwarzanie danych obciążenia (konwersja na MWh)...
3. Przetwarzanie danych cenowych...
4. Formatowanie kolumn (Date, Month, Year, ID)...

--- PODGLĄD DANYCH (Pierwsze 5 wierszy) ---
| date                      |   month |   year |   period_id |   res_share |   fossil_share |   market_price |   price_volatility |   energy_consumption |   system_stability |   crisis_flag |
|:--------------------------|--------:|-------:|------------:|------------:|---------------:|---------------:|-------------------:|---------------------:|-------------------:|--------------:|
| 2015-01-31 00:00:00+00:00 |       1 |   2015 |           1 |    0.208388 |       0.240069 |        31.0469 |           13.3287  |          4.54631e+07 |          0.0848555 |             0 |
| 2015-02-28 00:00:00+00:00 |       2 |   2015 |           2 |    0.170218 |       0.278148 |        36.6942 |           12.5827  |          4.23894e+07 |        